In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
import scipy
from scipy.sparse import hstack
from scipy.special import logit, expit
from sklearn.svm import SVC

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [2]:
df_train = pd.read_csv('../input/train.csv').fillna(' ')
df_test = pd.read_csv('../input/test.csv').fillna(' ')

In [3]:
df = pd.concat([df_train, df_test], 0)
nrow_train = df_train.shape[0]
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
df['loglen'] = np.log1p(df['comment_text'].str.len())
loglen = scipy.sparse.csr_matrix(df['loglen'].values)

In [5]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=10000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=20000)
words = word_vectorizer.fit_transform(df['comment_text'])
chars = char_vectorizer.fit_transform(df['comment_text'])

In [6]:
X = scipy.sparse.hstack((words, 
                         chars,
                         loglen.transpose())).tocsr()

In [7]:
X_train = X[:nrow_train]
X_test = X[nrow_train:]

In [ ]:
#testing
losses_log = []
losses_nb = []
for class_name in class_names:
    train_target = df_train[class_name]
    classifier_log = LogisticRegression(solver='sag')

    cv_loss_log = np.mean(cross_val_score(classifier_log, X_train, train_target, cv=3, scoring='roc_auc'))
    losses_log.append(cv_loss_log)
    print('Logit CV score for class {} is {}'.format(class_name, cv_loss_log))

print('Total CV score is {}'.format(np.mean(losses_log + losses_nb)))

In [10]:
predictions = {'id': df_test['id']}
for class_name in class_names:
    train_target = df_train[class_name]
    classifier_log = LogisticRegression(solver='sag')

    classifier_log.fit(X_train, train_target)
    predictions[class_name] = classifier_log.predict_proba(X_test)[:,1]

KeyboardInterrupt: 

In [ ]:
submission = pd.DataFrame.from_dict(predictions)
submission.to_csv('preds.csv', index=False)